# A64 - minimal timespans

In [1]:
import hyp3_sdk as sdk
import asf_search as asf
import geopandas as gpd
import pandas as pd
import os
import copy

In [2]:
hyp3 = sdk.HyP3() # 'https://hyp3-api.asf.alaska.edu'
print('User:{user_id} Quota:{quota}'.format(**hyp3.my_info()))

User:scottyhq Quota:{'max_jobs_per_month': 3000, 'remaining': 3000}


In [3]:
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          86.870901, 
          27.904352
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)

In [4]:
results = asf.search(
                    platform=[asf.PLATFORM.SENTINEL1],
                    processingLevel=asf.BURST,
                    beamMode=asf.BEAMMODE.IW,
                    #relativeOrbit=relorb,
                    polarization='VV',
                    #flightDirection='ASCENDING',
                    intersectsWith=str(gfa.geometry.values[0]),
                    #start='2018-01-01',
                    #end='2023-09-01',
                    #season=[180,270], #first,last day of year June~30*6=180
                    )
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
print(len(gf))
# Useful to fine burstIDs of interest

34


In [5]:
gf['burstId'] = gf.burst.str['fullBurstID']
gf.groupby('burstId').sceneName.count()

burstId
012_023790_IW1    18
121_258661_IW2    16
Name: sceneName, dtype: int64

In [6]:
# Pick a single burst ID to work with
burst = '012_023790_IW1'

In [7]:
# Look at timeline of acquisition
gfb = gf[gf.burstId == burst]
gfb['datetime'] = pd.to_datetime(gfb.startTime)

/Users/scott/miniforge3/envs/hyp3/lib/python3.12/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
gfb.datetime.describe()

count                                     18
mean     2023-10-01 12:14:31.541781504+00:00
min         2023-06-21 12:14:27.949790+00:00
25%      2023-08-11 12:14:31.018858752+00:00
50%      2023-10-01 12:14:33.413763584+00:00
75%      2023-11-21 12:14:32.349478400+00:00
max         2024-01-11 12:14:29.979483+00:00
Name: datetime, dtype: object

In [9]:
gfb.datetime.diff(-1).describe()

count                            17
mean     12 days 00:00:00.119393705
std       0 days 00:00:00.638915326
min         11 days 23:59:59.209060
25%         11 days 23:59:59.607499
50%         12 days 00:00:00.075485
75%         12 days 00:00:00.569214
max         12 days 00:00:01.230987
Name: datetime, dtype: object

In [10]:
subset = gfb.set_index('datetime')
#subset = subset[subset.index.month == 7] 
# Lots of selection opytion - random, first, last,e tc
#subset = subset.groupby([subset.index.year, subset.index.month]).first()
len(subset.fileID.to_list())

18

In [11]:
subset['date'] = pd.to_datetime(subset.startTime) #.dt.strftime('%Y%m%d')

In [12]:
job_definition = {
  "name": None,
  "job_type": "INSAR_ISCE_BURST",
  "job_parameters": {
      "granules": [None,None],
      "apply_water_mask": True, #default=False not enabled?
      "looks": "5x1", # '20x4', '10x2', '5x1' default=20x4
  },
}
job_definition

{'name': None,
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': [None, None],
  'apply_water_mask': True,
  'looks': '5x1'}}

In [13]:
# create interferogram job definitions
jobs = [] # List of job dictionaries

#for i in [0,1,2]: 
for i in range(len(subset)-1):
    jobdef = copy.deepcopy(job_definition) #Avoid overwriting in-memory!
    
    ref = subset.iloc[i]
    sec = subset.iloc[i+1]
    jobname = f"{burst}_{ref.date.strftime('%Y%m%d')}_{sec.date.strftime('%Y%m%d')}_5rlks"

    jobdef['name'] = jobname
    jobdef['job_parameters']['granules'] = [ref.fileID, sec.fileID]
    jobs.append(jobdef)

In [15]:
print(len(jobs))
# NOTE: each job should complete in ~20minutes
jobs[0]

17


{'name': '012_023790_IW1_20240111_20231230_5rlks',
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': ['S1_023790_IW1_20240111T121428_VV_5827-BURST',
   'S1_023790_IW1_20231230T121429_VV_B9CD-BURST'],
  'apply_water_mask': True,
  'looks': '5x1'}}

## Submit!


In [16]:
jobs = hyp3.submit_prepared_jobs(jobs)

In [17]:
# monitor jobs
batch = hyp3.find_jobs(job_type='INSAR_ISCE_BURST')

In [18]:
# NOTE: up to 85 running simultaneously
df = pd.DataFrame([x.to_dict() for x in batch])
df = df[df.name.str.startswith(burst)]
df.status_code.value_counts()

status_code
PENDING    17
Name: count, dtype: int64